In [93]:
import simtk.openmm.app as app
import simtk.openmm as mm
import simtk.unit as u

import mdtraj
import numpy as np

import parmed as pmd

## System

In [82]:
prmtop = app.AmberPrmtopFile('complex.top')
pdb = app.PDBFile('complex.pdb')
system = prmtop.createSystem(nonbondedMethod=app.PME, 
                             constraints=app.HBonds, 
                             nonbondedCutoff=12*u.angstroms, 
                             switchDistance=10*u.angstroms)
topology = mdtraj.Topology.from_openmm(prmtop.topology)


total_steps = 10000 # Reducing for testing purposes 4000000

## Integrator, forces and restraints

In [83]:
integrator = mm.LangevinIntegrator(50*u.kelvin, 1/u.picosecond, 0.002*u.picoseconds)

In [84]:
barostat = mm.MonteCarloBarostat(1.0*u.bar, 300*u.kelvin)

In [5]:
atoms_to_restrain = topology.select('not water and not type H')
default_k = 4.0*u.kilocalories_per_mole/u.angstroms**2

harmonic_restraint = mm.CustomExternalForce("k*((x-x0)^2+(y-y0)^2+(z-z0)^2)")
harmonic_restraint.addGlobalParameter('k', default_k)
harmonic_restraint.addPerParticleParameter("x0")
harmonic_restraint.addPerParticleParameter("y0")
harmonic_restraint.addPerParticleParameter("z0")


2

In [72]:
atoms_to_restrain = topology.select('not water and not type H')

## Init simulation and positions

In [6]:
for atomindex in atoms_to_restrain:
    position = pdb.positions[atomindex]
    harmonic_restraint.addParticle(int(atomindex), position.value_in_unit_system(u.md_unit_system))

system.addForce(harmonic_restraint)

5

In [85]:
simulation = app.Simulation(prmtop.topology, system, integrator)

simulation.context.setPositions(pdb.positions)
simulation.context.setVelocitiesToTemperature(50*u.kelvin)

In [97]:
a = pmd.load_file('complex.top', 'complex.pdb')

In [98]:
pmd.openmm.energy_decomposition(a, simulation.context)

{'bond': -81778.19281181952, 'total': -81778.19279602532}

In [99]:
s = simulation.context.getSystem()

In [100]:
s.getForces()


[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x618900420> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x613693180> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x6136931e0> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x613693240> >,
 <simtk.openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x61513e2a0> >]

In [104]:
u.nanometers

Unit({BaseUnit(base_dim=BaseDimension("length"), name="nanometer", symbol="nm"): 1.0})

## Minimize energy
100 steps while gradually releasing the constraints then
1000 steps with no constraints

In [35]:
for scaled_k in default_k*10*np.logspace(0, 10, num=11, base=0.5):
    simulation.context.setParameter('k', scaled_k)
    simulation.minimizeEnergy(maxIterations=1)
    
simulation.context.setParameter('k', 0)
simulation.minimizeEnergy(maxIterations=10)

## Heating

In [9]:
state = simulation.context.getState(getPositions=True)
positions = state.getPositions()

for restraint in range(harmonic_restraint.getNumParticles()):
    atomindex, _ = harmonic_restraint.getParticleParameters(restraint)
    position = positions[atomindex]
    harmonic_restraint.setParticleParameters(restraint ,atomindex, position.value_in_unit_system(u.md_unit_system))

harmonic_restraint.updateParametersInContext(simulation.context)
simulation.context.setParameter('k', default_k)

In [4]:
for temperature in np.linspace(50, 300, 251)*u.kelvin:
    integrator.setTemperature(temperature)
    simulation.step(10)
    print(simulation.context.getState(getEnergy=True).getKineticEnergy(),
          simulation.context.getState(getEnergy=True).getPotentialEnergy(),
          integrator.getTemperature())
    
simulation.step(5000)

## Equilibrate

In [13]:
simulation.system.addForce(barostat)

6

In [20]:
state = simulation.context.getState(getPositions=True)
positions = state.getPositions()

for restraint in range(harmonic_restraint.getNumParticles()):
    atomindex, _ = harmonic_restraint.getParticleParameters(restraint)
    position = positions[atomindex]
    harmonic_restraint.setParticleParameters(restraint ,atomindex, position.value_in_unit_system(u.md_unit_system))

harmonic_restraint.updateParametersInContext(simulation.context)

In [21]:
for scaled_k in default_k*10*np.logspace(0, 10, num=11, base=0.5):
    simulation.context.setParameter('k', scaled_k)
    simulation.step(50000)
    
simulation.context.setParameter('k', 0)
simulation.step(470000)

simulation.saveState('equilibrated.xml')

## Production

In [41]:
simulation.reporters.append(CheckpointReporter('checkpnt.chk', 200000))
simulation.reporters.append(DCDReporter('simulation.dcd', 5000))
simulation.reporters.append(StateDataReporter('simulation.log', 10000 ,
                                              step=True, 
                                              time=True, 
                                              totalEnergy=True))

In [39]:
simulation.step(2000000)